References :
1> Campuswire : https://campuswire.com/c/G06C55090/feed/432 
2> Campuswire : https://campuswire.com/c/G06C55090/feed/458
3> Lecture: https://liangfgithub.github.io/Notes/lec_W7.2_HMM.pdf

Assignment : https://liangfgithub.github.io/Coding/Coding4.html

## Hints

https://campuswire.com/c/G06C55090/feed/612
Hi All,

I'm struggling in debugging coding4 part2 Baum-Welch algorithm. I first calculated alpha (dim [T-1, mz]), beta (dim [T, mz]), gamma (dim [T-1, mz, mz]), and margiGamma (dim [T, mz]), then I updated the A (mz-by-mz) and B (mz-by-mx) according to the hints in #458. Following is my resulted alpha, beta, A, and B for the first iteration. The results are inaccurate and I have no idea which part went wrong. Has anyone met the same issue? Any help or hints will be appreciated!

I finally found the bug that when I read the observations I used "myData <-read.csv("https://liangfgithub.github.io/Data/coding4_part2_data.txt", header = True)" which omitted the first observation. After change header to FALSE, I can get the correct A and B now.
It is also helpful to run the R baumWelch "myfit2 = baumWelch(tmphmm, myData[,1], 2)" to debug with desired iteration numbers.

## Implementation

In [1]:
import numpy as np

In [2]:
# Load the data sequence from the file
data = np.genfromtxt("coding4_part2_data.txt", dtype=int)
w = np.array([0.5, 0.5])
A = np.array([[0.5, 0.5], [0.5, 0.5]])
B = np.array([[1/9, 3/9, 5/9], [1/6, 2/6, 3/6]])

In [3]:
def compute_forward_probability( w, A, B, X):
    T = len(X)  # Get the length of the sequence X
    mz = A.shape[0]
    # Initialize the forward probability matrix
    alpha = np.zeros((T, mz))
    
    # Calculate the forward probabilities recursively
    for t in range(T):
        if t == 0:
            # At t = 1, use the initial distribution w and emission probability B
            alpha[t] = w * B[:, X[t]-1]
            #print(f" t: {t} , alpha[t] : {alpha[t] }")
        else:
            # For t > 1, calculate alpha using the previous time step's alpha, transition probability A, and emission probability B
            for i in range(mz):
                alpha[t, i] = np.sum(alpha[t - 1] * A[:, i]) * B[i, X[t]-1]
                #print(f" t: {t} , i:{i}, alpha[{t},{i}] : {alpha[t,i] }")
    
    return alpha


In [4]:
def compute_backward_probability(t, w, beta, A, B, x):

    if t < 0:
        return 1

    else:
        
        # Recursive case: When t < T
        summation = 0
        if t == len(x):
            
            # Base case: When t is the last time step
            beta[t-1] =1
            return compute_backward_probability(t - 1, w, beta, A, B, x)
        else:
            #print(f"when t not 200 => t:{t}, beta[t+1] = beta[{t}+1] = beta[{t}]: {beta[t]}")
            t = (t-1)
           
            
            if t < 0:
                return 1
            
            for i in range(w.shape[0]):
                
                
                for j in range(A.shape[0]):

                    
                    summation += A[i][j] * B[j][x[t]-1] * beta[t+1][j]
                    


                # call recursively by updating the value of t
                beta[t][i] = summation  # Store the calculated value in the beta matrix
                
                
            
            #t = (t+1) # convert the index to row number before sending to recorsive function
            return compute_backward_probability(t , w, beta, A, B, x)
            




In [5]:
def calculate_myGamma(data,w,A,B, alpha,beta):
    mz = A.shape[0]
    T = data.shape[0]
    print(f"mz:{mz}, T:{T}, alpha : {alpha.shape} , beta : {beta.shape}")
    
    myGamma = np.zeros((mz, mz, T-1))
    print(myGamma.shape)
    # Formula my Gamma  = Summation (alpha_t_i * A(i,j) * beta(t+1, j))
    
    # iterate over T
    for t in range(T-1):
        # calculate gamma values for each pair of hidden states (i,j)
        
        for i in range(mz):
            
            for j in range(mz):
                gamma_t_i_j = alpha[t][i] * A[i][j] * B[j][data[t+1] - 1] * beta[t+1][j]
                myGamma[i][j][t] =gamma_t_i_j
            
    return myGamma

In [6]:
def compute_gamma_onestep(data, w,A,B):
    T = len(data)
    wz = w.shape[0]
    #print(f"T:{T}, wz:{wz}")
    
    alpha_one_step = compute_forward_probability(w,A,B,data)
    print(f"alpha_one_step : {alpha_one_step.shape} , alpha_one_step: {alpha_one_step[1:10]}")

    # Initialize the beta array
    beta_one_step = np.zeros((len(data), A.shape[0]))
    
    #print(f"beta_one_step_shape: {beta_one_step.shape} ")
    #print(f"data: {data}")
    compute_backward_probability(T,w ,beta_one_step,A,B, data)
    #compute_backward_probability(len(x1), w1, beta1, A1, B1, x1)
    #print("β{}({}) = {}".format(T, wz, beta_t_i))
    print(f"beta_one_step: {beta_one_step.shape} , beta_one_step\n-----\n{beta_one_step[190:,]} ")
    
    
    ### CALCULATE myGamma
    myGamma = calculate_myGamma(data, w,A,B,alpha_one_step, beta_one_step)
    print(f"myGamma-shape: {myGamma.shape} , ") #myGamma\n-----\n{myGamma[1]} 
    
    return myGamma
    ### Now As we have alpha & beta -> Calculate gamma one step

In [7]:
def update_A(A, B , gamma):
    mz, mz, t = gamma.shape
    # Perform the iterative operation
    A_updated = np.zeros_like(A)
    for i in range(mz):
        for j in range(mz):
            denominator = 0
            for j_prime in range(mz):
                for time_step in range(t - 1):
                    denominator += gamma[i, j_prime, time_step]

            for time_step in range(t - 1):
                numerator = 0
                for j_prime in range(mz):
                    numerator += gamma[i, j_prime, time_step]

                A_updated[i, j] = numerator / denominator
    return A_updated

In [8]:
def update_B( B , gamma):
    mz, mz, t = gamma.shape
    mx = B.shape[1]
    # Perform the iterative operation
    B_updated = np.zeros_like(B)
    
    # Calculate the updated B matrix
    for i in range(mz):
        for l in range(mx):
            #print(f" i : {i} , l :{l}")
            numerator = gamma[:, :, l].sum()  # Sum over t where x(t) = l
            denominator = gamma[i, :, :].sum()  # Sum over t

            # Avoid division by zero by replacing zeros in the denominator with 1
            if denominator == 0:
                denominator = 1

            B_updated[i, l] = numerator / denominator
    
    return B_updated

In [9]:
def execute_M_Step_onestep(data, w,A,B, myGamma):
    updated_A = update_A(A,B, myGamma)
    updated_B = update_B(B, myGamma)
    return updated_A,updated_B

In [10]:
def BW_onestep(data, w,A,B):
    
    # STEP 1: E step
    myGamma = compute_gamma_onestep(data, w,A,B)
    # Step 2: M Step
    updated_A, updated_B = execute_M_Step_onestep(data, w,A,B, myGamma)
    
    print("----updated A -------")
    print(updated_A)
    print("----updated B -------")
    print(updated_B)
    # return updated parameters A, B
    return updated_A, updated_B

In [11]:
BW_onestep(data, w, A, B)

alpha_one_step : (200, 2) , alpha_one_step: [[0.09259259 0.08333333]
 [0.04886831 0.04398148]
 [0.02579161 0.02321245]
 [0.01361224 0.01225101]
 [0.00718424 0.00646581]
 [0.00379168 0.00341251]
 [0.00040023 0.00060035]
 [0.00027794 0.00025015]
 [0.00014669 0.00013202]]
beta_one_step: (200, 2) , beta_one_step
-----
[[0.00217828 0.00435655]
 [0.00435655 0.0087131 ]
 [0.00560128 0.01120256]
 [0.02520576 0.05041152]
 [0.05041152 0.10082305]
 [0.06481481 0.12962963]
 [0.12962963 0.25925926]
 [0.16666667 0.33333333]
 [0.33333333 0.66666667]
 [1.         1.        ]] 
mz:2, T:200, alpha : (200, 2) , beta : (200, 2)
(2, 2, 199)
myGamma-shape: (2, 2, 199) , 
----updated A -------
[[6.85701315e-37 6.85701315e-37]
 [7.27519180e-37 7.27519180e-37]]
----updated B -------
[[0.64498375 0.43766755 0.29698869]
 [0.68431843 0.46435894 0.31510071]]


(array([[6.85701315e-37, 6.85701315e-37],
        [7.27519180e-37, 7.27519180e-37]]),
 array([[0.64498375, 0.43766755, 0.29698869],
        [0.68431843, 0.46435894, 0.31510071]]))

# # test 100 iterations

In [12]:
import numpy as np
import pandas as pd

/Users/amritpandey/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [13]:
n = 100

data_test = np.squeeze(pd.read_csv('coding4_part2_data.txt', header=None).values)
w_test = np.array([0.5, 0.5])
A_test = np.array([[0.5, 0.5], [0.5, 0.5]])
B_test = np.array([[1/9, 3/9, 5/9], [1/6, 2/6, 3/6]])
for i in range (100):
    print(f"======= Start {i} =======" )
    A_test,B_test = BW_onestep(data_test, w_test, A_test, B_test)
    print(f"======= End {i} =======" )

    

<ipython-input-7-57888127ecc2>:17: RuntimeWarning: invalid value encountered in double_scalars
  A_updated[i, j] = numerator / denominator


======= Start 0 =======
alpha_one_step : (200, 2) , alpha_one_step: [[0.09259259 0.08333333]
 [0.04886831 0.04398148]
 [0.02579161 0.02321245]
 [0.01361224 0.01225101]
 [0.00718424 0.00646581]
 [0.00379168 0.00341251]
 [0.00040023 0.00060035]
 [0.00027794 0.00025015]
 [0.00014669 0.00013202]]
beta_one_step: (200, 2) , beta_one_step
-----
[[0.00217828 0.00435655]
 [0.00435655 0.0087131 ]
 [0.00560128 0.01120256]
 [0.02520576 0.05041152]
 [0.05041152 0.10082305]
 [0.06481481 0.12962963]
 [0.12962963 0.25925926]
 [0.16666667 0.33333333]
 [0.33333333 0.66666667]
 [1.         1.        ]] 
mz:2, T:200, alpha : (200, 2) , beta : (200, 2)
(2, 2, 199)
myGamma-shape: (2, 2, 199) , 
----updated A -------
[[6.85701315e-37 6.85701315e-37]
 [7.27519180e-37 7.27519180e-37]]
----updated B -------
[[0.64498375 0.43766755 0.29698869]
 [0.68431843 0.46435894 0.31510071]]
======= End 0 =======
======= Start 1 =======
alpha_one_step : (200, 2) , alpha_one_step: [[9.47303707e-038 1.00507553e-037]
 [4.10075

 [nan nan]]
beta_one_step: (200, 2) , beta_one_step
-----
[[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [ 1.  1.]] 
mz:2, T:200, alpha : (200, 2) , beta : (200, 2)
(2, 2, 199)
myGamma-shape: (2, 2, 199) , 
----updated A -------
[[nan nan]
 [nan nan]]
----updated B -------
[[nan nan nan]
 [nan nan nan]]
======= End 33 =======
======= Start 34 =======
alpha_one_step : (200, 2) , alpha_one_step: [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
beta_one_step: (200, 2) , beta_one_step
-----
[[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [ 1.  1.]] 
mz:2, T:200, alpha : (200, 2) , beta : (200, 2)
(2, 2, 199)
myGamma-shape: (2, 2, 199) , 
----updated A -------
[[nan nan]
 [nan nan]]
----updated B -------
[[nan nan nan]
 [nan nan nan]]
======= End 34 =======
======= Start 35 =======
alpha_one_step : (200, 2) , alpha_one_step: [[nan nan]
 

 [nan nan]]
beta_one_step: (200, 2) , beta_one_step
-----
[[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [ 1.  1.]] 
mz:2, T:200, alpha : (200, 2) , beta : (200, 2)
(2, 2, 199)
myGamma-shape: (2, 2, 199) , 
----updated A -------
[[nan nan]
 [nan nan]]
----updated B -------
[[nan nan nan]
 [nan nan nan]]
======= End 66 =======
======= Start 67 =======
alpha_one_step : (200, 2) , alpha_one_step: [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
beta_one_step: (200, 2) , beta_one_step
-----
[[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [ 1.  1.]] 
mz:2, T:200, alpha : (200, 2) , beta : (200, 2)
(2, 2, 199)
myGamma-shape: (2, 2, 199) , 
----updated A -------
[[nan nan]
 [nan nan]]
----updated B -------
[[nan nan nan]
 [nan nan nan]]
======= End 67 =======
======= Start 68 =======
alpha_one_step : (200, 2) , alpha_one_step: [[nan nan]
 

 [ 1.  1.]] 
mz:2, T:200, alpha : (200, 2) , beta : (200, 2)
(2, 2, 199)
myGamma-shape: (2, 2, 199) , 
----updated A -------
[[nan nan]
 [nan nan]]
----updated B -------
[[nan nan nan]
 [nan nan nan]]
======= End 96 =======
======= Start 97 =======
alpha_one_step : (200, 2) , alpha_one_step: [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
beta_one_step: (200, 2) , beta_one_step
-----
[[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [ 1.  1.]] 
mz:2, T:200, alpha : (200, 2) , beta : (200, 2)
(2, 2, 199)
myGamma-shape: (2, 2, 199) , 
----updated A -------
[[nan nan]
 [nan nan]]
----updated B -------
[[nan nan nan]
 [nan nan nan]]
======= End 97 =======
======= Start 98 =======
alpha_one_step : (200, 2) , alpha_one_step: [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]
beta_one_step: (200, 2) , beta_one_step
-----
[[nan nan]
 [nan nan]
 

In [14]:
print("----updated A -------")
print(A_test)
print("----updated B -------")
print(B_test)

----updated A -------
[[nan nan]
 [nan nan]]
----updated B -------
[[nan nan nan]
 [nan nan nan]]
